In [0]:
# docs = {}

# with open(path + "FiQA_train_doc_final.tsv",'r') as f:
#     for line in f:
#         # [qid, doc_id, rank]
#         line = line.strip().split('\t')

#         if len(line) == 4:
#             docid = int(line[1])
#             doc = line[2]

#             docs[docid] = []
#             docs[docid].append(doc)

# # take(5, docs.items())

In [0]:
path = "drive/My Drive/FiQA/"

# Doc ID to Doc text
collection = pd.read_csv(path + "FiQA_train_doc_final.tsv", sep="\t")
collection = collection[['docid', 'doc']]
collection = collection.sort_values(by=['docid'])

# Question ID and Question text
query_df = pd.read_csv(path + "FiQA_train_question_final.tsv", sep="\t")

queries = query_df[['qid', 'question']]

all_answers_id = collection['docid'].to_list()
#print(all_answers_id)

# Question ID and Answer ID pair
qid_docid = pd.read_csv(path + "FiQA_train_question_doc_final.tsv", sep="\t")

qid_docid = qid_docid [['qid', 'docid']]

# Cleaning data
empty_docs = []
empty_id = []

for index, row in collection.iterrows():
    if pd.isna(row['doc']):
        empty_docs.append(row['docid'])
        empty_id.append(index)

# print(empty_docs)
print(empty_id)

# Remove empty answers
collection2 = collection.drop(empty_id)

# Answers Tokenization
collection2['doc_processed'] = collection2['doc'].apply(pre_process)
collection2['tokenized_ans'] = collection2.apply(lambda row: wordpunct_tokenize(row['doc_processed']), axis=1)
collection2['ans_len'] = collection2.apply(lambda row: len(row['tokenized_ans']), axis=1)

collection2.head(5)

len(collection2)

avg_ans_count = collection2['ans_len'].mean()

print(avg_ans_count)

# Questions Tokenization
queries = queries.copy()
queries['q_processed'] = queries['question'].apply(pre_process)
queries['tokenized_q'] = queries.apply(lambda row: wordpunct_tokenize(row['q_processed']), axis=1)
queries['q_len'] = queries.apply(lambda row: len(row['tokenized_q']), axis=1)

queries.head(5)

avg_q_count = queries['q_len'].mean()

print(avg_q_count)

# collection2.to_csv("processed_ans.csv", index=None)
# queries.to_csv("processed_q.csv", index=None)

In [0]:
# word2index = {"PAD": 0}
# word2count = {}

# idx = 1

# for index, row in collection2.iterrows():
#     for word in row['tokenized_ans']:
#         if word not in word2index:
#             word2index[word] = idx
#             idx += 1
#             word2count[word] = 1
#         else:
#             word2count[word] += 1
            
# ans_vocab_size = len(word2index)

# print("Answer vocab size: {}".format(ans_vocab_size))

# idx = len(word2index)

# for index, row in queries.iterrows():
#     for word in row['tokenized_q']:
#         if word not in word2index:
#             word2index[word] = idx
#             idx += 1
#             word2count[word] = 1
#         else:
#             word2count[word] += 1

# print(len(word2index))

# q_vocab_size = len(word2index) - ans_vocab_size
# print("Question vocab size: {}".format(q_vocab_size))

In [0]:
# # Reduce the size of the vocabuary
# word2idx = {"PAD": 0}
# word2c = {}
# idx = 1

# for word, count in word2count.items():
#     if count > 3:
#         if word not in word2idx:
#             word2idx[word] = idx
#             idx += 1
#             word2c[word] = count

# print(word2idx)

In [0]:
# # Vocab size
# len(word2idx)

# c = Counter(word2c)
# mc = c.most_common(30)

In [0]:
# with open('vocab_full.pickle', 'wb') as handle:
#     pickle.dump(word2index, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('vocab.pickle', 'wb') as handle:
#     pickle.dump(word2idx, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('vocab_count.pickle', 'wb') as handle:
#     pickle.dump(word2c, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # Label to answer text
# docid_tokenized = collection2[['docid', 'tokenized_ans']]

# docid_tokenized.head(5)

# label_to_ans = {}

# for index, row in docid_tokenized.iterrows():
#     label_to_ans[row['docid']] = row['tokenized_ans']

# print(take(5, label_to_ans.items()))

# with open('label_ans.pickle', 'wb') as handle:
#     pickle.dump(label_to_ans, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
# # Question to question text
# q_tokenized = queries[['qid', 'tokenized_q']]

# qid_to_text = {}

# for index, row in q_tokenized.iterrows():
#     qid_to_text[row['qid']] = row['tokenized_q']

# print(take(5, qid_to_text.items()))

# with open('qid_text.pickle', 'wb') as handle:
#     pickle.dump(qid_to_text, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
# Answer Ranking for each question
doc_ranking = pd.read_csv(path + "run_train_small.tsv", sep="\t", header=None)
doc_ranking = doc_ranking.rename(columns={0: 'qid', 1: 'doc_id', 3:'rank'})

# Create dict for query id and ranked candidates
# key: query ids, values: list of 1000 ranked candidates
qid_ranked_docs = {}

with open(path + "run_train_small.tsv",'r') as f:
    for line in f:
        # [qid, doc_id, rank]
        line = line.strip().split('\t')
        qid = int(line[0])
        doc_id = int(line[1])
        rank = int(line[2])
        
        if qid not in qid_ranked_docs:
            # Create a list of size 1000 for each query to store the candidates
            candidates = [0]*100
            qid_ranked_docs[qid] = candidates
        qid_ranked_docs[qid][rank-1] = doc_id
        
print(take(1, qid_ranked_docs.items()))

save_pickle('qid_ranked_docs_100.pickle', qid_ranked_docs)

In [0]:
# qid_rel = {}

# for index, row in qid_docid.iterrows():
    
#     if row['qid'] not in qid_rel:
#         qid_rel[row['qid']] = []
#     qid_rel[row['qid']].append(row['docid'])
    
# with open('qid_rel.pickle', 'wb') as handle:
#     pickle.dump(qid_rel, handle, protocol=pickle.HIGHEST_PROTOCOL)